<a href="https://colab.research.google.com/github/ShoaibSheriff/Computer-Vision/blob/master/Action%20Spotting/Direction_Baseline_Drink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms

import torch
import torch.nn as nn

from pprint import pprint

from PIL import Image
import numpy as np
import scipy.misc as sm
import imageio

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt

In [0]:
import torchvision.models as models
resnet34 = models.resnet34(pretrained=True)
resnet_binary = nn.Sequential(resnet34, nn.Sequential(nn.Linear(1000,2), nn.Softmax()))

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
import glob, os
os.chdir("/home/shoaibsheriff/Edu/CSE_527/Project/frames/")
train_ids = []
for file in glob.glob("*.mp4"):
    train_ids.append(file[:-4])
# print(train_ids)

In [0]:
df_ava_train = pd.read_csv(r"/home/shoaibsheriff/Edu/CSE_527/Project/ava_train_v1.0.csv", names  = ['video_id', 'middle_frame_timestamp', 'x1', 'y1', 'x2', 'y2', 'action_id'], index_col = False)

In [0]:
df_drink = df_ava_train.loc[((df_ava_train['action_id'] == 27) & (df_ava_train['video_id'].isin(train_ids)))]

In [0]:
df_drink

,video_id,middle_frame_timestamp,x1,y1,x2,y2,action_id
869,-FaXLcSFjUI,1084,0.332,0.363,0.517,0.855,27
875,-FaXLcSFjUI,1096,0.039,0.074,0.939,1.000,27
877,-FaXLcSFjUI,1099,0.150,0.016,0.906,0.997,27
881,-FaXLcSFjUI,1105,0.199,0.006,0.972,1.000,27
2222,0f39OWEqJ24,994,0.588,0.102,0.908,0.985,27
...,...,...,...,...,...,...,...
166754,yMtGmGa8KZ0,1660,0.069,0.161,0.277,0.727,27
166757,yMtGmGa8KZ0,1660,0.400,0.052,0.964,0.970,27
166804,yMtGmGa8KZ0,1693,0.027,0.147,0.287,0.870,27
166812,yMtGmGa8KZ0,1696,0.051,0.191,0.276,0.671,27


In [0]:
MIN = df_drink['middle_frame_timestamp'].min()
MAX = df_drink['middle_frame_timestamp'].max()

In [0]:
videos = {}

for video in df_drink['video_id'].unique():
  # print(video)
  df_drink_v = df_drink.loc[(df_drink['video_id'] == video)]
  tss = df_drink_v['middle_frame_timestamp'].to_list()

  closest = {}
  left = []
  right = []

  last_left = 11111111
  for ts in range(MIN, MAX+1):
    if ts in tss:
      last_left = ts
    if (ts-1 in tss) & (ts+2 in tss):
      last_left = ts
    if (ts-2 in tss) & (ts+1 in tss):
      last_left = ts
    closest[ts] = ts - last_left

  last_right = 11111111
  for ts in range(MAX, MIN-1, -1):
    if ts in tss:
      last_right = ts
    if (ts-1 in tss) & (ts+2 in tss):
      last_right = ts
    if (ts-2 in tss) & (ts+1 in tss):
      last_right = ts

    if (abs(closest[ts]) > last_right-ts):
      closest[ts] = ts - last_right
      
  for key in closest.keys():
    if closest[key] > 0:
      left.append(key)
    if closest[key] < 0:
      right.append(key)
  videos[video] = [left, right]

  # print(left)
  # print(right)

In [0]:
data = {}
data['timestamps'] = []
data['video'] = []
data['label'] = []

for video in df_drink['video_id'].unique():

  [left, right] = videos[video]

  for ts in left:
    data['timestamps'].append(ts)
    data['video'].append(str(video) + ".mp4")
    data['label'].append(0)

  for ts in right:
    data['timestamps'].append(ts)
    data['video'].append(str(video) + ".mp4")
    data['label'].append(1)

print(len(data['timestamps']))
print(len(data['video']))
print(len(data['label']))

53218
53218
53218


In [0]:
# video_cache = {}

from torch.utils.data import DataLoader,sampler,Dataset

class ActionSearchDataset(Dataset):

    def __init__(self, act_training_data):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.act_training_data = act_training_data

    def __len__(self):
        return len(self.act_training_data['label'])
        return len(videos)

    def __getitem__(self, idx):
      
        videos = self.act_training_data['video']
        timestamps = self.act_training_data['timestamps']
        labels = self.act_training_data['label']
        
#         print(videos[idx])
#         print(timestamps[idx])

        frame = imageio.imread('/home/shoaibsheriff/Edu/CSE_527/Project/frames/'+videos[idx]+'/'+str("{0:0=3d}".format(timestamps[idx]-904))+'.jpg')
#         plt.imshow(frame)
#         plt.show()

        train_transforms = transforms.Compose([
          transforms.ToPILImage(),
          transforms.Resize(255),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        frame2 = train_transforms(frame)
#         plt.imshow(frame2)
#         plt.show()

        # plt.imshow(frame)
        # plt.show()

#         print(frame.shape)
        label = labels[idx]
        sample = {'image': frame2, 'label': label}

        return sample

In [0]:
dataset=ActionSearchDataset(data)

for i in range(10):
    sample=dataset[i]
    print(sample['image'].shape)
    print(sample['label'])

torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0


In [0]:
batches = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=1)

In [0]:
for i,sample in enumerate(batches):
  print(i,sample['image'].shape, " ", sample['label'])
  if i>10: 
      break

0 torch.Size([32, 3, 224, 224])   tensor([0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 1, 0, 0, 1, 0, 1])
1 torch.Size([32, 3, 224, 224])   tensor([0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 1, 1, 0, 1])
2 torch.Size([32, 3, 224, 224])   tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 1, 1, 0, 0, 1, 0])
3 torch.Size([32, 3, 224, 224])   tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1])
4 torch.Size([32, 3, 224, 224])   tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 0, 1, 0, 0, 0, 0])
5 torch.Size([32, 3, 224, 224])   tensor([1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 1, 0, 1])
6 torch.Size([32, 3, 224, 224])   tensor([1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
    

In [0]:
def check_accuracy(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for t, sample in enumerate(loader):
        
        x_var = Variable(sample['image'].float().cuda())
        y_var = Variable(sample['label'].long().cuda())
        
        y_var=y_var.cpu()
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
#         print(preds)
#         print(y_var)
        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [0]:
def train(model, loss_fn, optimizer, dataloader, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        loss1 = 0.0
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['image'].float().cuda())
            y_var = Variable(sample['label'].long().cuda())

            scores = model(x_var)
            loss =loss_fn(scores, y_var)
            loss1 += loss.data
            if (t + 1) % 100 == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss1))
                loss1 = 0
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        check_accuracy(resnet_binary, batches)

In [0]:
import copy
gpu_dtype = torch.cuda.FloatTensor

resnet_binary = copy.deepcopy(resnet_binary)
resnet_binary.cuda()

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [0]:
loss_fn = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(resnet_binary.parameters(), lr = 1e-4)

torch.cuda.random.manual_seed(12345)
resnet_binary.train() 
train(resnet_binary, loss_fn, optimizer, batches, num_epochs=50) 

Starting epoch 1 / 50


/home/shoaibsheriff/anaconda3/lib/python3.7/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


t = 100, loss = 67.0063
t = 200, loss = 61.0331
t = 300, loss = 59.5261
t = 400, loss = 59.4777
t = 500, loss = 56.4915
t = 600, loss = 56.5413
t = 700, loss = 56.4089
t = 800, loss = 54.2475
t = 900, loss = 53.5017
t = 1000, loss = 52.7835
t = 1100, loss = 52.5097
t = 1200, loss = 51.4717
t = 1300, loss = 50.8809
t = 1400, loss = 50.2705
t = 1500, loss = 49.7542
t = 1600, loss = 50.9631
Got 43644 / 53218 correct (82.01)
Starting epoch 2 / 50
t = 100, loss = 48.7469
t = 200, loss = 48.1699
t = 300, loss = 47.5921
t = 400, loss = 47.5151
t = 500, loss = 47.7377
t = 600, loss = 46.4003
t = 700, loss = 46.4351
t = 800, loss = 46.4549
t = 900, loss = 46.1918
t = 1000, loss = 47.5186
t = 1100, loss = 47.7971
t = 1200, loss = 46.8685
t = 1300, loss = 46.7577
t = 1400, loss = 46.7207
t = 1500, loss = 47.5680
t = 1600, loss = 45.3137
Got 46209 / 53218 correct (86.83)
Starting epoch 3 / 50
t = 100, loss = 44.5504
t = 200, loss = 44.2444
t = 300, loss = 45.5733
t = 400, loss = 44.5086
t = 500, l

KeyboardInterrupt: 

In [0]:
# torch.save(resnet_binary.state_dict(), "act4.h5")
optimizer = optim.Adam(resnet_binary.parameters(), lr = 5e-5)
train(resnet_binary, loss_fn, optimizer, batches, num_epochs=10) 

Starting epoch 1 / 10
t = 100, loss = 39.6372
t = 200, loss = 38.6755
t = 300, loss = 37.8852
t = 400, loss = 37.6654
t = 500, loss = 37.7873
t = 600, loss = 37.9184
t = 700, loss = 37.1325
t = 800, loss = 37.1179
t = 900, loss = 36.8195
t = 1000, loss = 37.1010
t = 1100, loss = 37.7774
t = 1200, loss = 37.2455
t = 1300, loss = 37.6392
t = 1400, loss = 37.4842
t = 1500, loss = 36.9609
t = 1600, loss = 37.2299
Got 50878 / 53218 correct (95.60)
Starting epoch 2 / 10
t = 100, loss = 36.6612
t = 200, loss = 36.4082
t = 300, loss = 35.4859
t = 400, loss = 36.3070
t = 500, loss = 36.1893
t = 600, loss = 35.8015
t = 700, loss = 35.5354
t = 800, loss = 36.7835
t = 900, loss = 35.8013
t = 1000, loss = 36.3274
t = 1100, loss = 36.4730
t = 1200, loss = 36.5130
t = 1300, loss = 36.7744
t = 1400, loss = 36.2267
t = 1500, loss = 35.7337
t = 1600, loss = 36.2952
Got 51304 / 53218 correct (96.40)
Starting epoch 3 / 10
t = 100, loss = 36.0865
t = 200, loss = 35.4035
t = 300, loss = 35.4399
t = 400, los

In [0]:
torch.save(resnet_binary, "/home/shoaibsheriff/Edu/CSE_527/Project/model_20.h5")

# Test

In [0]:
import pandas as pd

In [0]:
model_final = torch.load("/home/shoaibsheriff/Edu/CSE_527/Project/model_20.h5")
model_final.eval()

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [0]:
import glob, os
os.chdir("/home/shoaibsheriff/Edu/CSE_527/Project/test_videos/")
test_ids = []
for file in glob.glob("*.mp4"):
    test_ids.append(file[:-4])

In [0]:
df_test = pd.read_csv(r"/home/shoaibsheriff/Edu/CSE_527/Project/ava_test_v1.0.csv",names=['video_id','middle_frame_timestamp', 'x1','y1','x2','y2', 'action_id'], header=None)

In [0]:
df_test_dr_valid = df_test.loc[((df_test['action_id'] == 27) & (df_test['video_id'].isin(test_ids)))]

In [0]:
MIN = df_test_dr_valid['middle_frame_timestamp'].min()
MAX = df_test_dr_valid['middle_frame_timestamp'].max()+1

In [0]:
video_ids = df_test_dr_valid['video_id'].unique()

In [0]:
video_ids

array(['7YpF6DntOYw', 'BCiuXAuCKAU', 'HKjR70GCRPE', 'IzvOYVMltkI',
       'LgBQlW6OTr0', 'P60OxWahxBQ', 'g1wyIcLPbq0', 'kMy-6RtoOVU',
       'yn9WN9lsHRE', 'z-fsLpGHq6o', 'zR725veL-DI'], dtype=object)

In [0]:
def get_next_guess(cap, guess_time):
    
    cap.set(cv2.CAP_PROP_POS_MSEC,guess_time*1000) 
    ret, frame = cap.read()
    
#     plt.imshow(frame)
#     plt.show()
    
    train_transforms = transforms.Compose([
      transforms.ToPILImage(),
      transforms.Resize(255),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    frame2 = train_transforms(frame)
    frame3 = frame2.unsqueeze(0)
#     print(frame3.shape)
    scores = model_final(frame3.cuda())
    _, preds = scores.data.max(1)
    
    if(preds.cpu().numpy()) == 0:
        if (guess_time == MIN):
            return -1
        next_guess = random.randint(MIN, guess_time)
    else:
        if (guess_time == MAX):
            return -1
        next_guess = random.randint(guess_time, MAX)
        
    return next_guess

In [0]:
import random

def find_action(dataf, video) :    
    
    search_num = 1
    guess_time = random.randint(MIN, MAX)
    
    cap = cv2.VideoCapture('/home/shoaibsheriff/Edu/CSE_527/Project/test_videos/'+video+'.mp4')


    for i in range(500) :
        if guess_time in dataf:
            break;
        if (guess_time - 1 in dataf) & (guess_time + 2 in dataf):
            break;
        if (guess_time - 2 in dataf) & (guess_time + 1 in dataf):
            break;
        else :
            search_num = search_num + 1
        
        guess_time = get_next_guess(cap, guess_time)
        if (guess_time == -1):
            return 500
    
    return search_num

In [0]:
total = 0
ct = 0
for i in range(100) : 
    print('epoch', i)
    for video_id in video_ids:
        print(video_id)
        tss = df_test_dr_valid.loc[(df_test_dr_valid['video_id'] == video_id)]['middle_frame_timestamp'].to_list()
    #     print(tss)

        as_metric = 500
        while (as_metric == 500) :
            as_metric = find_action(tss, video_id)

        total = total + as_metric
        ct = ct + 1
#         print('avg', total/ct)
print('avg', total/ct)

epoch 0
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 1
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 2
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 3
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 4
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 5
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 6
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 7
7YpF6DntOYw


HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 59
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 60
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 61
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 62
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 63
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 64
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
IzvOYVMltkI
LgBQlW6OTr0
P60OxWahxBQ
g1wyIcLPbq0
kMy-6RtoOVU
yn9WN9lsHRE
z-fsLpGHq6o
zR725veL-DI
epoch 65
7YpF6DntOYw
BCiuXAuCKAU
HKjR70GCRPE
I